In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from scipy.stats import uniform, randint

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [9]:
df = pd.read_csv('Данные для построения модели, задача - Регрессия для IC50.csv')
df

,Chi0n,HallKierAlpha,Chi0,BCUT2D_CHGHI,EState_VSA10,Target feature,Kappa1,AvgIpc,SMR_VSA4,fr_allylic_oxid,...,BCUT2D_LOGPLOW,VSA_EState5,EState_VSA7,EState_VSA3,MolLogP,VSA_EState8,NumValenceElectrons,MinAbsPartialCharge,EState_VSA2,Target Feature
0,19.534409,-0.66,20.208896,2.600532,0.000000,1,20.606247,3.150503,43.480583,0,...,-2.322229,1.764908,0.000000,21.659962,7.1212,16.981087,158,0.038844,0.000000,0.795141
1,19.794682,-0.08,20.208896,2.614066,0.000000,0,21.163454,3.150503,33.495774,0,...,-2.444817,1.914377,0.000000,21.659962,6.1556,17.670565,162,0.012887,0.000000,-0.112478
2,23.689110,-0.08,23.794682,2.665274,0.000000,0,25.026112,3.214947,33.495774,0,...,-2.565224,1.930720,0.000000,21.659962,7.1292,18.287216,186,0.094802,0.000000,2.349877
3,20.241516,-0.66,20.916003,2.600529,0.000000,1,21.567454,3.179270,43.480583,0,...,-2.322030,1.769975,0.000000,21.659962,7.5113,17.012013,164,0.038844,0.000000,0.231883
4,22.617677,-2.22,24.639617,2.602486,0.000000,0,23.194917,3.337074,43.480583,0,...,-2.318893,1.605178,0.000000,10.829981,9.1148,14.692318,184,0.062897,10.829981,2.029917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,18.825334,-1.65,21.637464,2.726237,14.383612,0,20.263719,3.023764,51.752408,2,...,-2.646743,-0.405436,0.000000,11.835812,4.3002,8.824371,164,0.317890,52.409521,1.491363
997,21.810933,-2.18,25.499271,2.725543,19.178149,0,24.511583,3.130958,51.752408,2,...,-2.655659,-1.300354,0.000000,5.917906,3.8049,10.378794,192,0.327562,52.347395,1.505149
998,23.633394,-1.83,27.620591,2.725818,19.178149,0,27.726151,3.204255,51.752408,2,...,-2.656447,-0.487671,6.255769,12.338728,4.5381,8.774745,210,0.327887,46.932405,1.491360
999,23.380977,-2.45,27.921921,2.730109,23.972686,0,29.111081,2.887043,51.752408,0,...,-2.678553,-6.848660,6.923737,19.262465,3.3649,7.488627,208,0.312509,30.512100,1.505136


In [10]:
X, y = df.drop('Target Feature', axis=1), df['Target Feature']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Target Feature', axis=1), df['Target Feature'], test_size=0.3, random_state=42)

#### Контрольной метрика для задачи регрессии будем считать R^2 score. Она будет записана в атрибут assessment_criterion BestRegressionModel класса

In [12]:
class BestRegressionModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели регресии
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'R^2',
            'value': 0.5
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LinearRegression(),
                'best_model': None,
                'best_params': None,
                'params': {},
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestRegressor(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': ElasticNet(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'alpha': [0.01, 0.1, 1, 10],
                    'l1_ratio': [0.2, 0.5, 0.8]
                },
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVR(),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'MSE': '',
                    'MAE': '',
                    'R^2': ''
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            self.models[model]['metrics']['MSE'] = mean_squared_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['MAE'] = mean_absolute_error(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['R^2'] = r2_score(y, self.models[model]['data']['y_pred'])
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"MSE: {self.models[model]['metrics']['MSE']:.4f}")
            print(f"MAE: {self.models[model]['metrics']['MAE']:.4f}")
            print(f"R^2: {self.models[model]['metrics']['R^2']:.4f}")
            print()
            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return 10 ** result

#### Подбор гипперпараметров для моделей

In [13]:
best_regression_model = BestRegressionModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LinearRegression() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели LinearRegression(): {}

Подбор гипперпараметров для модели RandomForestRegressor(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestRegressor(random_state=42): {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 9, 'n_estimators': 181}

Подбор гипперпараметров для модели ElasticNet(random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели ElasticNet(random_state=42): {'alpha': 0.01, 'l1_ratio': 0.2}

Подбор гипперпараметров для модели SVR() методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию параме

#### Поиск лучшей модели

In [14]:
best_regression_model.get_best_model()

Обучаем модель LinearRegression() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LinearRegression():
MSE: 0.4693
MAE: 0.5288
R^2: 0.4696

Обучаем модель RandomForestRegressor(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestRegressor(random_state=42):
MSE: 0.3488
MAE: 0.4506
R^2: 0.6058

Обучаем модель ElasticNet(random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели ElasticNet(random_state=42):
MSE: 0.4492
MAE: 0.5256
R^2: 0.4923

Обучаем модель SVR() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандарт

#### Формирование предсказания лучшей моделью

In [15]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestRegressor(max_depth=20, min_samples_leaf=2, min_samples_split=9,
                      n_estimators=181, random_state=42)
Предсказание сформировано


array([5.25766935e+02, 2.88416796e+02, 1.62001502e+01, 2.73488757e+01,
       1.84627377e+02, 1.03421064e+02, 5.64151884e+02, 2.46042965e+02,
       2.44886794e-01, 4.44669116e+02, 2.32573276e+01, 1.23265445e+01,
       1.50992859e+01, 2.07702939e+01, 5.39202263e+01, 3.22553307e+00,
       5.60488148e+01, 6.06584474e-02, 5.41093828e+02, 1.18820318e+01,
       1.05350176e+02, 1.85966225e+01, 8.23559698e+00, 4.18053540e+02,
       2.79583910e+02, 1.98797003e+01, 1.13011062e+03, 7.78667542e+01,
       4.89449691e+00, 2.44886794e-01, 1.15006026e+01, 1.20324642e+01,
       3.67722772e+02, 1.93144846e+02, 6.82616890e+01, 4.54624118e+01,
       4.21799330e+02, 2.36778709e+01, 2.51101506e+01, 1.68629701e+01,
       8.35731490e+00, 1.68629701e+01, 2.45128493e+01, 7.57473754e+01,
       2.40318138e+01, 4.18066972e+01, 5.26615493e+01, 2.24022380e+00,
       1.26634646e+02, 3.12047412e+02, 1.19879537e+01, 5.38215486e-01,
       5.42141364e+02, 7.83047631e+01, 6.40490975e+02, 5.46498751e+02,
      